# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes from this dataset:  
https://huggingface.co/datasets/huuuyeah/meetingbank

I've put them in my Google Drive. The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this to work, you'll also need to download a clip from the Dever City Council, or for more fun, record something of your own!

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00


In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [8]:
# New capability - connect this Colab to my Google Drive

drive.mount("/content/drive")
# audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"
audio_filename = "/audio case llms - Made with Clipchamp.mp4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Audio File

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [9]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [10]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [11]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Once he was taken into custody for that forgery charge, he was transported to the Altoona Police Department. Once at the Altoona Police Department, that male was searched instant to arrest. It was discovered that he had a United States passport that identified him as Luigi Mangione. From there, continued search of his property, a firearm was located and a suppressor was also located. The New York Post reported that Luigi Mangione had documents with him that were critical of the health insurance industry. Pennsylvania Governor Josh Shapiro condemned those who have been celebrating Brian Thompson's murder online. Brian Thompson was a father to two. He was a husband and he was a friend to many. And yes, he was the CEO of a health insurance company. In America, we do not kill people in cold blood to resolve policy differences or express a viewpoint. I understand people have real frustration 

In [12]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Arrest Details in United Healthcare CEO Murder Case. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [13]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [14]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Arrest Details in United Healthcare CEO Murder Case. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Once he was taken into custody for that forgery charge, he was transported to the Altoona Police Department. Once at the Altoona Police Department, that male was searched instant to arrest. It was discovered that he had a United States passport that identified him as Luigi Mangione. From there, continued search of his prop

In [15]:
response = tokenizer.decode(outputs[0])

In [16]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Arrest Details in United Healthcare CEO Murder Case. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Diolch am wylio'r fideo. Once he was taken into custody for that forgery charge, he was transported to the Altoona Police Department. Once at the Altoona Police Department, that male was searched instant to arrest. It was discovered that he had a United States passport that identified him as Luigi Mangione. From there, continued search of his property, a firearm was located and a suppressor was also located. The New York Post reported that Luigi Mangione had documents with him that were critical of the health insurance industry. Pennsylvania Governor Josh Shapiro condemned those who have been celebrating Brian Thompson's murder online. Brian Thompson was a father to two. He was a husband and he was a friend to many. And yes, he was the CEO of a health insurance company. In America, we do not kill people in cold blood to resolve policy differences or express a viewpoint. I understand people have real frustration with our health care system. And I have worked to address that throughout my career. But I have no tolerance, nor should anyone, for one man using an illegal ghost gun to murder someone because he thinks his opinion matters most. In a civil society, we are all less safe. Mangione is 26 years old. He has a bachelor's and a master's degree from the University of Pennsylvania. He had majored in computer science. I want to tell you about Upside. It's a free app that gets you cash back on things like gas and restaurants. This is real cash back. It's money that appears in your Upside app that you can transfer straight into your bank account. I used Upside when I went to Dunkin' Donuts for a cup of tea. I claimed an offer for Dunkin' on the app, paid as usual using a card, and got cash back. It's so easy. You can also use Upside at places like Shell, Exxon, 7-Eleven, Taco Bell, and that's just to name a few. To download Upside and start earning, click the link in the description or scan the QR code on your screen and use the promo code CRIMEFIX to get an extra 25 cents back on every gallon on your first tank of gas. That's promo code CRIMEFIX for extra cash back. I want to bring in Dave Sarney. He's a retired NYPD detective. Dave, your first thoughts on this arrest. It's exactly what you expect to happen. When you have law enforcement put all merits to bear, rubber to the road, the work that was done to get this person at least a photo of him was immense. It was a massive amount of work being done to go through all that video surveillance, the footage, and the like. And to have it culminate to this apprehension, and eventually it looks like they're going to charge him with the murder, it's what you expected with police department to do. When we look for someone, they're going to find them. The McDonald's, the person at the McDonald's is what really gets me. I mean, that is just so impressive to me. This person saw who they believed to be, the person in the photos, the person with the medical mask, the surgical mask, and those distinctive eyebrows, and of course the eyes. I mean, that's what got me when I saw the latest photograph that was released on Saturday was the eyes and the eyebrows. And so this person did the absolute right thing and called law enforcement. Law enforcement responded. They asked the person to take the mask down. They asked for ID. They said it was a fake ID. They went from there, and they're saying they found a ghost gun on him, which is incredibly disturbing because that's something that you could just make with a 3D printer. Absolutely. That's the other thing. Expecting the fact that you were possibly committed a murder and you keep the weapon with you, that's very telling in the fact that either he couldn't make a gun again. It was something he treasured a little bit. It operated, I guess, to what he thought it would. And was there anything beyond what he did on Wednesday morning? Was he thinking anything further than that? Because when you look at the manifesto, and apparently they're just doing a little digging in him, his anger toward corporations and the health care system was very vitriol, and that's something to be concerned about. But I will tell you, the amount of work that was put in to get that photo out and to have someone call, that's why detectives, good old-fashioned police work, sometimes, even though all the technology is out there, it trumps it. And that video, that photo, thankfully somebody did say something. And it's great that they found somebody in Altoona, Pennsylvania, who saw it and called in. And that is what is so amazing to me. With social media, with YouTube, all of these different ways that we communicate these days and they put information out, obviously somebody saw the photo and they called. Had that person not called, had they not seen the photo, this person, the suspect, he doesn't sound like he's formally charged just yet, would still be on the loose. Yeah, that's the thing. Like I said, this person is dangerous. That's the reality. He committed a murder, cold-blooded, calculated assassination of an individual, regardless of ideology or anything. He killed him in cold blood. He's out in the streets of New York. Then it looks like he gets on a bus. The benefit of the technology out there to be able to track him from pre-shooting to post-shooting and thereafter is beneficial. But again, if it's not the public that's involved, no matter how many photos we have, how many stills we have out there, how many videos out there, if the public does not engage in helping the police apprehend people like this, murderers, it's much harder in law enforcement to do. Law enforcement said that this investigation is not over. There are still a lot of questions. How did the shooter know that Brian Thompson would be in the location where he was? And they're still looking into whether or not there were potential accomplices here. What is your gut telling you on that? I mean, I don't know if I could get the schedule of a UnitedHealthcare CEO and know that he would be out on the street at 644 in the morning on a Wednesday. I mean, where could he have gotten this information? Well, that's the big question. Was this public information that was out there that there was a conference going to be held and he was going to be speaking at it? That's something that I think police are still looking at as to how he knew that and why then and there. He could have done it any time while he was in New York. He waited to that moment to do it. How did he know he was going to be there at that time? He went there early enough to wait for him so he knows he's going to be leaving the hotel or leaving where he is. So how did he get this information? I guarantee you they're looking into that also. As far as accomplices or people who participated or helped him in this, that's why you're going to scrub his social media accounts. You're going to do a background of everywhere he has been since he was apparently in Philadelphia and then went to Altoona. Everything along those lines, anything he's had, any cell phones that he had that he used, anything that he's had communications with is going to be looked at. As I said, it's not over. It isn't over because you're 26 years old. You get a gun. Did you make it yourself? And if not, who helped you make it? Not only that, who helped you figure out where this person was going to be? The suspect was going to be at the time.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Minutes**
====================

**Summary**
-----------

* Date: Today
* Location: Altoona Police Department
* Attendees: Josh Shapiro, Dave Sarney
* Purpose: Discussion on the arrest of Luigi Mangione in connection with the murder of Brian Thompson, CEO of UnitedHealthcare

**Discussion Points**
--------------------

* Luigi Mangione, a 26-year-old with a background in computer science, was arrested in connection with the murder of Brian Thompson
* Mangione was found with a ghost gun and a suppressor, and had documents critical of the health insurance industry
* The New York Post reported that Mangione had a history of anger towards corporations and the health care system
* The investigation is ongoing, with police looking into how Mangione obtained information about Thompson's schedule and potential accomplices
* Dave Sarney, a retired NYPD detective, commended the public for their involvement in helping the police apprehend the suspect
* Sarney emphasized the importance of good old-fashioned police work and the role of technology in solving the case

**Takeaways**
------------

* The arrest of Luigi Mangione marks a significant development in the investigation into the murder of Brian Thompson
* The case highlights the importance of public engagement and cooperation with law enforcement in solving crimes
* The investigation is ongoing, with police looking into potential accomplices and how Mangione obtained information about Thompson's schedule

**Action Items**
----------------

* **Owner:** Josh Shapiro
* **Action Item:** Continue to provide support and resources to the Altoona Police Department in their investigation into the murder of Brian Thompson
* **Owner:** Dave Sarney
* **Action Item:** Scrub Luigi Mangione's social media accounts and conduct a thorough background check on his activities since his arrival in Philadelphia
* **Owner:** Altoona Police Department
* **Action Item:** Investigate potential accomplices and determine how Luigi Mangione obtained information about Brian Thompson's schedule<|eot_id|>

## Alternative implementation

Class student Youssef has contributed this variation in which we use an open-source model to transcribe the meeting Audio.

Thank you Youssef!

In [20]:
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

In [21]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [23]:
audio_filename_opensource = "/6 Breaking Arrest Details in United Healthcare CEO Murder Case - Law&Crime Network (youtube) (1).mp3"

In [25]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename_opensource,return_timestamps=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [26]:
transcription = result["text"]
print(transcription)

 In some dark corners, this killer is being hailed as a hero. Hear me on this. He is no hero. The real hero in this story is the person who called 911 at McDonald's this morning. breaking details in the arrest of Luigi Mangione, the 26-year-old man police believe shot and killed UnitedHealthcare CEO Brian Thompson. It was a peaceful arrest, first and foremost. The officer quickly recognized the male as the suspect from the shooting, and within several minutes, seconds of the first contact, he had asked the suspect if he had been in New York City recently, and that really invoked a physical reaction from the suspect. He became visibly nervous, kind of shaking at that question. Welcome to Crime Fix. I'm Anjanette Levy. Luigi Mangione is in custody in Altoona, Pennsylvania. He is the prime suspect in the execution style killing of United Healthcare CEO, Brian Thompson in Midtown Manhattan less than a week ago. The arrest came after a five day long nationwide manhunt for a shooter whose na

In [27]:
display(Markdown(transcription))

 In some dark corners, this killer is being hailed as a hero. Hear me on this. He is no hero. The real hero in this story is the person who called 911 at McDonald's this morning. breaking details in the arrest of Luigi Mangione, the 26-year-old man police believe shot and killed UnitedHealthcare CEO Brian Thompson. It was a peaceful arrest, first and foremost. The officer quickly recognized the male as the suspect from the shooting, and within several minutes, seconds of the first contact, he had asked the suspect if he had been in New York City recently, and that really invoked a physical reaction from the suspect. He became visibly nervous, kind of shaking at that question. Welcome to Crime Fix. I'm Anjanette Levy. Luigi Mangione is in custody in Altoona, Pennsylvania. He is the prime suspect in the execution style killing of United Healthcare CEO, Brian Thompson in Midtown Manhattan less than a week ago. The arrest came after a five day long nationwide manhunt for a shooter whose name wasn't known until Monday. His face seen for only a brief moment when he dropped his mask during a visit to a hostel in New York City in November. After Brian Thompson was shot on a city street on December 4th, the search for the shooter began at 6.45 a.m. just after those shots were fired. The shooter fled on an electric bike into Central Park and then into a cab. Police said he then took a bus out of New York City. Saturday night, the NYPD released new photos showing the suspect wearing a surgical mask as he was getting into a cab. And then shortly after he entered, the camera in the cab capturing this crystal clear image of the man with the distinctive eyes and eyebrows. Then Monday morning, a major break. It came when a person called from a McDonald's in Altoona. The caller thought that he recognized a man in a mask. When officers arrived, they made contact with the male who had a face mask on. That mail was asked to remove his face mask and officers immediately recognized him as a suspect in the New York City shooting in recent days. That mail was asked for a form of identification. He then provided a fake identification card out of New Jersey. After a brief investigation, it was determined that the ID provided was in fact a fake ID and he was subsequently arrested for a forgery charge. Once he was taken into custody for that forgery charge, he was transported to the Altona Police Department. Once at the Altona Police Department, that mail was searched instant to arrest. It was discovered they had a United States passport that identified him as... Excuse me. Luigi Mangione. From there, continued search of his property. A firearm was located, and a suppressor was also located. The New York Post reported that Luigi Mangione had documents with him that were critical of the health insurance industry. Pennsylvania governor Josh Shapiro condemned those who have been celebrating Brian Thompson's murder online. Brian Thompson was a father to two. He was a husband and he was a friend to many. And yes, he was the CEO of a health insurance company. In America, we do not kill people in cold blood to resolve policy differences or express a viewpoint. I understand people have real frustration with our healthcare system, and I have worked to address that throughout my career. But I have no tolerance, nor should anyone, for one man using an illegal ghost gun to murder someone He murders someone because he thinks his opinion matters most. In a civil society, we are all less safe. Mangione is 26 years old. He has a bachelor's and a master's degree from the University of Pennsylvania. He had majored in computer science. I want to tell you about Upside. It's a free app that gets you cash back on things like gas and restaurants. This is real cash back. It's money that appears in your Upside app that you can transfer straight into your bank account. I used Upside when I went to Dunkin' Donuts for a cup of tea. I claimed an offer for Dunkin' on the app, paid as usual using a card and got cash back. It's so easy. You can also use Upside at places like Shell, Exxon, 7-Eleven, Taco Bell, and that's just to name a few. To download Upside and start earning, click the link in the description or scan the QR code on your screen and use the promo code crimefix to get an extra 25 cents back on every gallon on your first tank of gas. That's promo code crimefix or extra cash back. I want to bring in Dave Cerny. He's a retired NYPD detective. Dave, your first thoughts on this arrest. It's exactly what you expect to happen. When you have law enforcement put all, you know, all merits to bear, rubber to the road. The work that was done to get this person, and at least a photo of him was immense. It was a massive amount of work being done to go through all that video surveillance, the footage and the like. And to have it culminate to this apprehension and eventually it looks like they're gonna charge him with the murder. It's what you expected with police department to do. When we look for someone, they're gonna find them. The McDonald's, the person at the McDonald's is what really gets me. I mean, that is just so impressive to me. this person saw who they believed to be, the person in the photos, the person with the medical mask, the surgical mask, and those distinctive eyebrows, and of course the eyes. I mean, that's what got me when I saw the latest photograph that was released on Saturday was the eyes and the eyebrows. And so this person did the absolute right thing and called law enforcement. Law enforcement responded. They asked the person to take the mask down. They asked for ID, they said it was a fake ID, they went from there and they're saying they found a ghost gun on him, which is incredibly disturbing because that's something that you could just make with a 3D printer. Yeah, absolutely, that's the other thing. And, you know, expecting the fact that you just, you were possibly committed a murder and you keep the weapon with you, that says that's very telling in the fact that either he couldn't make a gun again, it was something he treasured a little bit, it operated, I guess, to what he thought it would. And was there anything beyond what he did on Wednesday morning? Was he thinking anything further than that? Because when you look at the manifesto, and apparently they're just doing a little digging on him, his anger toward corporations and the health care system was very vitriol. And that's something to be concerned about. But I will tell you, if the amount of work that was put in to get that photo out and to have someone call, That's why detectives, good old-fashioned police work, sometimes, even though all the technology's out there, it trumps it. And that video, that photo, thankfully somebody did say something. And it's great that they found somebody in Altoona, Pennsylvania who saw it and called it. That's, and that is what is so amazing to me. I mean, with social media, with YouTube, all of these different ways that we communicate these days and they put information out, obviously somebody saw the photo and they called. Had that person not called, had they not seen the photo, you know, this person, the suspect, he's not, doesn't sound like he's formally charged just yet, would still be on the loose. Yeah, that's the thing. He said, this person is dangerous. That's the reality. He committed a murder, cold blooded, calculated assassination of an individual, regardless of, you know, ideology or anything. He killed him in cold blood. He's going out, he's out in the streets of New York, then gets on, looks like he gets on a bus. And the benefit of the technology out there to be able to track him from pre-shooting to post-shooting and thereafter is beneficial. But again, if it's not the public that's involved, no matter how many photos we have, how many stills we have out there, how many videos out there, if the public does not engage in helping the police apprehend people like this, murderers, it's much harder as in law enforcement to do. Law enforcement said that this investigation is not over. There are still a lot of questions. How did the shooter know that Brian Thompson would be in the location where he was? And they're still looking into whether or not there were potential accomplices here. What is your gut telling you on that? I mean, I don't know if I could get the schedule of a UnitedHealthcare CEO and know that he would be out on the street at 644 in the morning on a Wednesday? I mean, where could he have gotten this information? Well, that's the big question, was this public information that was out there that there was a conference gonna be held that he was gonna be speaking at it? That's something that we're still, I think police are still looking at as to how he knew that and why then and there. He could have done it anytime while he was in New York. He waited to that moment to do it. How did he know he was gonna be there at that time? He went there early enough to wait for him so he knows he's going to be leaving the hotel or leaving where he is. So how do you get this information? That's, you know, that's what I guarantee you they're looking into that also. As far as accomplices or people who was participated to help them in this, that's why you're going to scrub his social media accounts. You're going to do a background of everywhere he's been since he was apparently in Philadelphia that went to Altoona. along those lines, anything he's had, any cell phones that he had that he used, anything that he had communications with is going to be looked at. As I said, it's not over. It isn't over because you're 26 years old. How do you get a gun? Did you make it yourself? And if not, who helped you make it? Not only that, who helped you figure out where this person was going to be, the suspect was going to be at the time, the suspect gets there at the time of the shooting, within that half hour time range. How did you know? Do you see this suspect eventually talking? Is this somebody that you see, you know, he's a well-educated guy. He's 26 years old. He has a master's degree from the University of Pennsylvania. He went to a very good school, very good private school, upscale private school in Maryland. Is this somebody who eventually wants to say something? I mean, if he's got writings about the health insurance industry on him, He obviously has a message he wants to put out there. Yeah, I was hoping that would happen, but apparently we're being told by the press conference that he no longer talking. So I don't know if he's not cooperating or he's a yes for an attorney. So once that happens, the attorney client privilege, that's over, he asked for a lawyer, then that's pretty much done. I would have thought he would have given the fact of what you see him doing, writing those words on those shell casings, the monopoly money on him could have been utilized, might have been something he wanted to use on after he shot the victim, throw him there. We don't know what he was gonna do with that. And the manifesto and all these things, you were hoping that, you know, you're 26, you wanna make a message or state your message. And given the fact, I guarantee you, he was probably reading all of the messages on social media, kind of praising what he did, committing a murder, I don't understand that. but if he has a message, I was hoping he would bring it out and he would talk, but it looks like that's not gonna happen in this case. Well, Dave Cerny, we'll be keeping a very close eye on it. Thank you as always. No, thank you very much for having me. Have a good night. And law enforcement is asking that anyone who has information about Luigi Mangione's movements to call them. That number is 804-PA-TIPS. And that's it for this episode of Crime Fix. I'm Ann Jeanette Levy. Thanks so much for being with me. I'll see you back here next time.